In [26]:
# resolving modules
import Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Flux")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [27]:
#getting modules
using DataFrames
using CSV
using Flux
using Statistics, LinearAlgebra

In [28]:
train_df = CSV.read("./data/train.csv", DataFrame)
display(describe(train_df))

size(train_df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,PassengerId,446.0,1,446.0,891,0,Int64
2,Survived,0.383838,0,0.0,1,0,Int64
3,Pclass,2.30864,1,3.0,3,0,Int64
4,Name,,"Abbing, Mr. Anthony",,"van Melkebeke, Mr. Philemon",0,String
5,Sex,,female,,male,0,String7
6,Age,29.6991,0.42,28.0,80.0,177,"Union{Missing, Float64}"
7,SibSp,0.523008,0,0.0,8,0,Int64
8,Parch,0.381594,0,0.0,6,0,Int64
9,Ticket,,110152,,WE/P 5735,0,String31


(891, 12)

Tratamento de dados

In [29]:
# coluna embarked possui poucas linhas nulas -> retiradas
# coluna age tem uma quantidade ok de linhas nulas -> trocar pela mediana
# coluna cabin tem muitas linhas nulas -> ignorar a coluna
train_df = dropmissing(train_df, "Embarked")
train_df.Age = replace(train_df.Age, missing => 28)
train_df = select(train_df, Not("Cabin"))

# passenger ID e nome são apenas identificadores e não ajudam a classificar
train_df = select(train_df, Not(["PassengerId", "Name"]))

describe(train_df)


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Survived,0.382452,0,0.0,1,0,Int64
2,Pclass,2.31159,1,3.0,3,0,Int64
3,Sex,,female,,male,0,String7
4,Age,29.3152,0.42,28.0,80.0,0,Float64
5,SibSp,0.524184,0,0.0,8,0,Int64
6,Parch,0.382452,0,0.0,6,0,Int64
7,Ticket,,110152,,WE/P 5735,0,String31
8,Fare,32.0967,0.0,14.4542,512.329,0,Float64
9,Embarked,,C,,S,0,String1


In [30]:
# verificando valores únicos de cada coluna não-numérica
combine(groupby(train_df,"Sex"), nrow=>"count")

Row,Sex,count
,String7,Int64
1,male,577
2,female,312


In [31]:
combine(groupby(train_df,"Ticket"), nrow=>"count")

Row,Ticket,count
,String31,Int64
1,A/5 21171,1
2,PC 17599,1
3,STON/O2. 3101282,1
4,113803,2
5,373450,1
6,330877,1
7,17463,1
8,349909,4
9,347742,3


In [32]:
combine(groupby(train_df,"Embarked"), nrow=>"count")

Row,Embarked,count
,String1,Int64
1,S,644
2,C,168
3,Q,77


In [33]:
# para Sex e Embarked, é possível codificar para numeros.
# para Ticket, são muitos valores únicos -> ignorar coluna

train_df.Sex = Int64.(replace(train_df.Sex, "female" => 1, "male" => 2))
train_df.Embarked = Int64.(replace(train_df.Embarked, "S" => 1, "C" => 2, "Q" => 3))

train_df = select(train_df, Not("Ticket"))
describe(train_df) # dataset limpo!

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,Survived,0.382452,0,0.0,1,0,Int64
2,Pclass,2.31159,1,3.0,3,0,Int64
3,Sex,1.64904,1,2.0,2,0,Int64
4,Age,29.3152,0.42,28.0,80.0,0,Float64
5,SibSp,0.524184,0,0.0,8,0,Int64
6,Parch,0.382452,0,0.0,6,0,Int64
7,Fare,32.0967,0.0,14.4542,512.329,0,Float64
8,Embarked,1.3622,1,1.0,3,0,Int64
